# 인코더

## 셀프 어텐션

**스케일드 점곱 어텐션**

In [1]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
inputs = tokenizer(text, return_tensors='pt', add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [3]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [4]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

torch.Size([1, 5, 768])

In [5]:
import torch
from math import sqrt

query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [6]:
scores

tensor([[[29.5569,  0.8675,  0.4436, -0.8891, -0.0672],
         [ 0.8675, 29.3100, -1.4127,  0.8812, -0.3837],
         [ 0.4436, -1.4127, 26.1765,  1.1114, -1.2986],
         [-0.8891,  0.8812,  1.1114, 28.7139, -3.0140],
         [-0.0672, -0.3837, -1.2986, -3.0140, 27.9385]]],
       grad_fn=<DivBackward0>)

In [8]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [10]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [13]:
weights.shape, value.shape

(torch.Size([1, 5, 5]), torch.Size([1, 5, 768]))

In [15]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpoze(1, 2)/sqrt(dim_k))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

**멀티 헤드 어텐션**